In [4]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Activation,BatchNormalization,Lambda,Input,Conv2D,Bidirectional,LSTM, Concatenate, MaxPooling2D,GlobalAveragePooling2D
import tensorflow.keras.backend as K
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from time import time
import datetime
import numpy as np
from generater import crnnGenerator
import logging

logger=tf.get_logger()
logger.setLevel(logging.ERROR)

In [5]:

def crnn():
    x=image_input=Input(name='image_input',shape=[128,64,1],dtype='float32')
    """
    minx=Input(name='minx',shape=[1],dtype='float32')
    miny=Input(name='miny',shape=[1],dtype='float32')
    maxx=Input(name='maxx',shape=[1],dtype='float32')
    maxy=Input(name='maxy',shape=[1],dtype='float32')
    def cropimage(img,minx=0,miny=0,maxx=0,maxy=0):
        
        return tf.image.crop_to_bounding_box(img,minx,miny,maxx-minx,maxy-miny)
    x=Lambda(cropimage)([x,minx,miny,maxx,maxy])
    
    """
        
    x=Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv1_1')(x)
    x=MaxPooling2D(pool_size=(2, 2), name='pool1', padding='same')(x)#64 32
    
    x=Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv2_1')(x)
    #x=MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool2', padding='same')(x)# 32 16

    x=Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_1')(x)
    x=Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_2')(x)
    #x=MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool3', padding='same')(x)# 16 8
    
    x=Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv4_1')(x)
    x=BatchNormalization(name='batch1')(x)
    
    x=Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv5_1')(x)
    
    x=BatchNormalization(name='batch2')(x)
    #x=MaxPooling2D(pool_size=(2, 2), strides=(1, 2), name='pool5', padding='valid')(x)# 8 4
    #63 31 512
    x=aa=Conv2D(512, (2, 2), strides=(1, 1), activation='relu', padding='valid', name='conv6_1')(x)#63 31 512
    x=keras.layers.Reshape((-1,512))(x)
    print(aa)
    x=Bidirectional(LSTM(256, return_sequences=True))(x)
    x=Bidirectional(LSTM(256, return_sequences=True))(x)
    
    num_classes=36+1
    x=keras.layers.Dense(num_classes, name='dense1')(x)#알파벳+숫자#1953 36
    
    x=y_pred= Activation('softmax', name='softmax')(x)#1953 36
    
   # model_pred=keras.models.Model(inputs=[image_input,minx,miny,maxx,maxy],outputs=x)
    model_pred=keras.models.Model(inputs=image_input,outputs=x)
    
    #maxstringlen=int(y_pred.shape[1])
    maxstringlen=100
    
    def ctc_lambda_func(args):
        y_pred,labels, input_length, label_length = args
          #y_pred = y_pred[:, 2:, :] 
        return K.ctc_batch_cost(y_true=labels,y_pred=y_pred,input_length=input_length,label_length=label_length)    
        #return tf.nn.ctc_loss(labels=labels,logits=y_pred,logit_length=input_length,label_length=label_length,logits_time_major=False)    
    
    labels = Input(name='labels', shape=[maxstringlen], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64') 
    
    ctcloss=Lambda(ctc_lambda_func,output_shape=(1,),name='ctcloss')([y_pred,labels,input_length,label_length])
    
   # model_train=keras.models.Model(inputs=[image_input,minx,miny,maxx,maxy,labels,input_length,label_length],outputs=ctc_loss)
    model_train=keras.models.Model(inputs=[image_input,labels,input_length,label_length],outputs=ctcloss)
    
    return model_train,model_pred,maxstringlen

    

In [6]:
traincrnn,predcrnn,inputlength=crnn()

KerasTensor(type_spec=TensorSpec(shape=(None, 63, 31, 512), dtype=tf.float32, name=None), name='conv6_1/Relu:0', description="created by layer 'conv6_1'")


In [7]:
predcrnn.load_weights('D:/ocr/checkpoints/202107180819_crnn_train_v1/weights.010.h5')

In [8]:
predcrnn.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 128, 64, 1)]      0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 128, 64, 64)       640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 64, 32, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 64, 32, 128)       73856     
_________________________________________________________________
conv3_1 (Conv2D)             (None, 64, 32, 256)       295168    
_________________________________________________________________
conv3_2 (Conv2D)             (None, 64, 32, 256)       590080    
_________________________________________________________________
conv4_1 (Conv2D)             (None, 64, 32, 512)       118016

In [9]:
import cv2

In [71]:
imgs=np.zeros((1,64,128))
img=cv2.imread('D:/ocr/abcd/'+'000adfe5b817011c_1.jpg',cv2.IMREAD_GRAYSCALE)
img=cv2.resize(img,(128,64))

img=img.astype(np.float32)
img=img/255.0

imgs[0,:,:]=img
cv2.imshow('ada',imgs[0,:,:])
cv2.waitKey()
cv2.destroyAllWindows()

In [73]:
b=predcrnn.predict(imgs)

In [58]:
len(imgs[0,:,:])

64

In [59]:
len(img)

64

In [63]:
cv2.imshow('ada',img)

In [35]:
def labels_to_text(labels):     
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
    return ''.join(list(map(lambda x: alphabet[int(x)], labels)))

In [37]:
labels_to_text(a[0])

TypeError: only size-1 arrays can be converted to Python scalars

In [74]:
b[0][2]

array([0.08999861, 0.01014736, 0.01437327, 0.0073495 , 0.02121754,
       0.00387196, 0.0078904 , 0.00119138, 0.0273886 , 0.00068483,
       0.00068527, 0.04457089, 0.03048134, 0.08415134, 0.03857205,
       0.00896987, 0.00083473, 0.08202626, 0.04343214, 0.01138874,
       0.05583779, 0.01196147, 0.0056564 , 0.0093128 , 0.0032975 ,
       0.00039635, 0.00212559, 0.00149765, 0.00101079, 0.00057607,
       0.00112412, 0.00128617, 0.00074874, 0.00062258, 0.00107017,
       0.00132641, 0.37292334], dtype=float32)

In [75]:
res=[]
for data in b[0]:
    listdata=list(data)
    res.append( listdata.index(max(listdata)))
res
resu=[]
for i,abc in enumerate(res) :
    if abc!=36:
        resu.append(abc)
resu

[0]

In [56]:
re

NameError: name 're' is not defined